<a href="https://colab.research.google.com/github/brendanpshea/data_clean_nypl/blob/main/New_York_Public_Library_Menus_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New York Public Library-Menus - Data Cleaning



Brendan's Notes:

The original data files are available here:

https://uofi.app.box.com/s/zh2hxfkq0cc6vyftw91nqa4smdpq7ybk

I just downloaded all the files (as a zip), and stuck them on dropbox (see below). From there, I just ran the standard Pandas commands to get an overview of the data.


In [57]:
import os

url = "https://www.dropbox.com/scl/fi/l8b5np5xoes57nr1hqhae/NYPL-menus.zip?rlkey=pak0ox3wae0x0yd09d23eoqma&st=mfq9dka9&dl=1"
output_file = "NYPL-menus.zip"

# Check if the file already exists
if not os.path.exists(output_file):
    !wget -q "$url" -O "$output_file"
    !unzip "NYPL-menus.zip"
else:
    print(f"{output_file} already exists. No download needed.")

!ls

NYPL-menus.zip already exists. No download needed.
dirty_menus.db	menu_clean.csv	    menupage_clean.csv	nypl_menus.db	sample_data
dish_clean.csv	menuitem_clean.csv  NYPL-menus		NYPL-menus.zip


## ERD Diagram
Here is an entity relationship diagram for the data.

In [58]:
import base64
from IPython.display import Image, display, HTML

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))


mm("""

erDiagram
    DISH ||--o{ MENUITEM : "is included in"
    MENU ||--o{ MENUPAGE : "contains"
    MENUPAGE ||--o{ MENUITEM : "includes"

    DISH {
        int id PK
        string name
        float description
        int menus_appeared
        int times_appeared
        int first_appeared
        int last_appeared
        float lowest_price
        float highest_price
    }

    MENU {
        int id PK
        string name
        string sponsor
        string event
        string venue
        string place
        string physical_description
        string occasion
        string notes
        string call_number
        float keywords
        float language
        string date
        string location
        float location_type
        string currency
        string currency_symbol
        string status
        int page_count
        int dish_count
    }

    MENUITEM {
        int id PK
        int menu_page_id FK
        float price
        float high_price
        float dish_id FK
        string created_at
        string updated_at
        float xpos
        float ypos
    }

    MENUPAGE {
        int id PK
        int menu_id FK
        string page_number
        string image_id
        string full_height
        string full_width
        string uuid
        string created_at
        string updated_at
    }
""")

### Load Data Using Pandas


In [59]:
import pandas as pd
import numpy as np

dish_df = pd.read_csv('NYPL-menus/Dish.csv')
menu_df = pd.read_csv('NYPL-menus/Menu.csv')
menuitem_df = pd.read_csv('NYPL-menus/MenuItem.csv')
menupage_df = pd.read_csv('NYPL-menus/MenuPage.csv')


## Main Use Case and Dirty Data Queries

We would like to see how the **popularity and price of different dishes have changed over the years**.  I'll use "spaghetti" as example here.

Let's run some queries for this using the (not yet cleaned!) data.

In [60]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('dirty_menus.db')

dish_df.to_sql('dishes', conn, if_exists='replace', index=False)
print("Dishes data loaded successfully.")

menu_df.to_sql('menus', conn, if_exists='replace', index=False)
print("Menus data loaded successfully.")

menuitem_df.to_sql('menuitems', conn, if_exists='replace', index=False)
print("Menu items data loaded successfully.")

menupage_df.to_sql('menupages', conn, if_exists='replace', index=False)
print("Menu pages data loaded successfully.")

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database created and data loaded successfully.")

# Reopen the connection to verify the data
conn = sqlite3.connect('nypl_menus.db')
cursor = conn.cursor()

# Check the number of rows in each table
tables = ['dishes', 'menus', 'menuitems', 'menupages']
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"Number of rows in {table} table: {count}")

# print table schema for all tables
for table in tables:
    cursor.execute(f"PRAGMA table_info({table})")
    schema = cursor.fetchall()
    print(f"\n\nTable Schema for {table}:")
    for column in schema:
        print(column)
conn.close()


Dishes data loaded successfully.
Menus data loaded successfully.
Menu items data loaded successfully.
Menu pages data loaded successfully.
Database created and data loaded successfully.
Number of rows in dishes table: 423397
Number of rows in menus table: 17545
Number of rows in menuitems table: 1332485
Number of rows in menupages table: 66937


Table Schema for dishes:
(0, 'id', 'INTEGER', 0, None, 0)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'menus_appeared', 'INTEGER', 0, None, 0)
(3, 'times_appeared', 'INTEGER', 0, None, 0)
(4, 'first_appeared', 'INTEGER', 0, None, 0)
(5, 'last_appeared', 'INTEGER', 0, None, 0)
(6, 'lowest_price', 'REAL', 0, None, 0)
(7, 'highest_price', 'REAL', 0, None, 0)


Table Schema for menus:
(0, 'id', 'INTEGER', 0, None, 0)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'sponsor', 'TEXT', 0, None, 0)
(3, 'event', 'TEXT', 0, None, 0)
(4, 'place', 'TEXT', 0, None, 0)
(5, 'physical_description', 'TEXT', 0, None, 0)
(6, 'occasion', 'TEXT', 0, None, 0)
(7, 'notes', 'TEXT', 0, No

In [61]:
%reload_ext sql
%sql sqlite:///dirty_menus.db

In [62]:
%%sql
SELECT COUNT(*) AS total_spaghetti_dishes
FROM dishes
WHERE name LIKE "%spaghetti%"

 * sqlite:///dirty_menus.db
   sqlite:///nypl_menus.db
Done.


total_spaghetti_dishes
1832


In [63]:
%%sql
SELECT name, COUNT(*) AS dish_count
FROM dishes
JOIN menuitems ON dishes.id = menuitems.dish_id
WHERE name LIKE "%spaghetti%"
GROUP BY menuitems.dish_id
ORDER BY dish_count DESC
LIMIT 15

 * sqlite:///dirty_menus.db
   sqlite:///nypl_menus.db
Done.


name,dish_count
Spaghetti,299
Spaghetti au Gratin,231
Spaghetti Italienne,180
Special Spaghetti with Fresh Mushrooms,119
Spaghetti a l'Italienne,113
"Spaghetti, Italienne",108
Spaghetti au gratin,90
Spaghetti Au Gratin,88
Spaghetti Milanaise,84
Spaghetti Bolognaise,55


In [64]:
%%sql
-- Get high-priced spagetti dishes
SELECT name, AVG(price) AS avg_price, COUNT() AS dish_count
FROM dishes
JOIN menuitems ON dishes.id = menuitems.dish_id
WHERE name LIKE "%spaghetti%"
GROUP BY menuitems.dish_id
ORDER BY avg_price DESC
LIMIT 15

 * sqlite:///dirty_menus.db
   sqlite:///nypl_menus.db
Done.


name,avg_price,dish_count
"Spaghettini alla ""Bassanese""",6000.0,1
Spaghettini al Pomodoro Fresco,4000.0,1
Spaghetti all'amatriciana,1000.0,1
Spaghetti alla tonnata,1000.0,1
Spaghetti alle vongole bianche,650.0,1
Les Spaghetti a l'Italienne,550.0,1
Spaghettie a la Bolognaise,550.0,1
Spaghettie with Meat Sauce,550.0,1
"Spaghetti ""Maitre d'Hotel""",500.0,1
Spaghetti all amatriciana,450.0,1


In [65]:
%%sql
-- Get earliest years for common spaghetti dishes
SELECT name, MIN(first_appeared) AS earliest_year
FROM dishes
JOIN menuitems ON dishes.id = menuitems.dish_id
WHERE name LIKE "%spaghetti%"
GROUP BY menuitems.dish_id
ORDER BY earliest_year ASC
LIMIT 10

 * sqlite:///dirty_menus.db
   sqlite:///nypl_menus.db
Done.


name,earliest_year
Spaghetti a la Bontout,0
Spaghetti alla Checca,0
Spaghettis Napolitaine,0
Spaghetti tomate et basilic,0
Spaghetti alla Certosina,0
Spaghetti with seafood sauce,0
Spaghetti con pancetta,0
Spaghetti a la (panse),0
Spaghetti with bacon,0
Spaghetti alle vongole bianche,0


In [66]:
%%sql
-- get latest years for common spaghetti dishes
SELECT name, MAX(last_appeared) AS latest_year
FROM dishes
JOIN menuitems ON dishes.id = menuitems.dish_id
WHERE name LIKE "%spaghetti%"
GROUP BY menuitems.dish_id
ORDER BY latest_year DESC
LIMIT 10

 * sqlite:///dirty_menus.db
   sqlite:///nypl_menus.db
Done.


name,latest_year
Special Spaghetti with Fresh Mushrooms,2928
Spaghetti,2928
Special home made spaghetti with veal ragu',2012
SPAGHETTI ALLA CHITARRA CON RAGU' DI CARNE DI VITELLO,2012
Spaghetti with sun-dried tomatoes,2012
SPAGHETTI CON POMODORI ESSICCATI AL SOLE,2012
Spaghetti Carbonara with pancetta and Parmesan,2006
Spaghetti Bolognese,2002
"MINI VEGI-LOAF Tofu, Chestnut and Cilantro Croquettes, Served over Spaghetti in a Tomato Sauce (Taro Spring Roll, Pickled Cabbage)",1999
SPAGHETTI MEAT BALLS,1999


## Data Cleaning
Now, let's clean the data. For each table, we will:

1. Get a profile of what it looks life before the query.
2. Clean it (to a new dataframe/csv.
3. Produce a profile of what it looks like after cleaning.

### Clean `dish_df`

In [67]:
dish_df.head()

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,1,Consomme printaniere royal,NaN,8,8,1897,1927,0.20,0.4
1,2,Chicken gumbo,NaN,111,117,1895,1960,0.10,0.8
2,3,Tomato aux croutons,NaN,13,13,1893,1917,0.25,0.4
3,4,Onion au gratin,NaN,41,41,1900,1971,0.25,1.0
4,5,St. Emilion,NaN,66,68,1881,1981,0.00,18.0


In [68]:
dish_df.shape

(423397, 9)

In [69]:
dish_df.describe().round(2)

,id,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
count,423397.00,0.0,423397.00,423397.00,423397.00,423397.00,394297.00,394297.0
mean,264456.59,NaN,3.06,3.15,1675.51,1679.30,0.97,1.6
std,150489.07,NaN,27.82,29.96,651.32,651.93,6.71,12.7
min,1.00,NaN,0.00,-6.00,0.00,0.00,0.00,0.0
25%,132374.00,NaN,1.00,1.00,1900.00,1900.00,0.00,0.0
50%,269636.00,NaN,1.00,1.00,1914.00,1917.00,0.00,0.0
75%,397135.00,NaN,1.00,1.00,1949.00,1955.00,0.40,0.6
max,515677.00,NaN,7740.00,8484.00,2928.00,2928.00,1035.00,3050.0


In [70]:
dish_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423397 entries, 0 to 423396
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              423397 non-null  int64  
 1   name            423397 non-null  object 
 2   description     0 non-null       float64
 3   menus_appeared  423397 non-null  int64  
 4   times_appeared  423397 non-null  int64  
 5   first_appeared  423397 non-null  int64  
 6   last_appeared   423397 non-null  int64  
 7   lowest_price    394297 non-null  float64
 8   highest_price   394297 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 29.1+ MB


In [71]:
dish_df.isnull().sum()

id                     0
name                   0
description       423397
menus_appeared         0
times_appeared         0
first_appeared         0
last_appeared          0
lowest_price       29100
highest_price      29100
dtype: int64

Main problems in `dish_df` include:

- The 'description' column is entirely null (423,397 null values)
- 'lowest_price' and 'highest_price' columns have 29,100 null values each
- Potential inconsistencies in casing, whitespace, and punctuation (not directly visible, but common issues)
- There are some extremely high prices.

Let's fix these using Pandas. First, let's introduce a function to deal with text issues (which come up repeatedly):

In [72]:
import re
def clean_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return text

    # Convert to lowercase
    text = text.lower()


    # Replace contractions
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)

    # Remove special characters, keeping only letters, numbers, and basic punctuation
    text = re.sub(r'[^a-zA-Z0-9\s.,!?()-]', '', text)

    # Standardize spacing around punctuation
    text = re.sub(r'\s*([.,!?()])\s*', r'\1 ', text)
    text = re.sub(r'\s+', ' ', text)

    # Remove extra periods
    text = re.sub(r'\.{2,}', '.', text)

    # Remove spaces at the start and end of parentheses and brackets
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\[\s+', '[', text)
    text = re.sub(r'\s+\]', ']', text)

    # Remove leading/trailing whitespace
    text = text.strip()

    # Capitalize first letter of each word (title case)
    text = text.title()

    return text

Now, we can apply this to clean our data.

In [73]:
df_without_null_columns = dish_df.dropna(axis=1, how='all')

def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (df[column] < lower_bound) | (df[column] > upper_bound) | (df[column] < 0)

df_outliers_handled = df_without_null_columns.copy()
for col in ['lowest_price', 'highest_price']:
    outliers = detect_outliers(df_outliers_handled, col)
    df_outliers_handled.loc[outliers, col] = np.nan

def impute_price(df, column):
    # First, try to impute using the median price for the year
    df[column] = df.groupby('first_appeared')[column].transform(lambda x: x.fillna(x.median()))

    # If there are still NaNs, fill with the overall median
    overall_median = df[column].median()
    df[column] = df[column].fillna(overall_median)

    return df[column]

df_imputed = df_outliers_handled.copy()
df_imputed['lowest_price'] = impute_price(df_imputed, 'lowest_price')
df_imputed['highest_price'] = impute_price(df_imputed, 'highest_price')

# Ensure highest_price is always >= lowest_price
df_imputed['highest_price'] = np.maximum(df_imputed['highest_price'], df_imputed['lowest_price'])

# years of 0 should be null
df_imputed['first_appeared'] = df_imputed['first_appeared'].replace(0, np.nan)
df_imputed['last_appeared'] = df_imputed['last_appeared'].replace(0, np.nan)

# years greater than current year should be null
current_year = pd.Timestamp('now').year
df_imputed['first_appeared'] = df_imputed['first_appeared'].replace(current_year + 1, np.nan)
df_imputed['last_appeared'] = df_imputed['last_appeared'].replace(current_year + 1, np.nan)

dish_clean_df = df_imputed.copy()
text_columns = ['name']  # Add other text columns if any
for col in text_columns:
    dish_clean_df[col] = dish_clean_df[col].apply(clean_text)

print("Null values after cleaning:")
print(dish_clean_df.isnull().sum())

print("\nShape of cleaned dataframe:", dish_clean_df.shape)

print("\nSample of cleaned data:")
print(dish_clean_df.head())

print("\nSummary statistics of price columns:")
print(dish_clean_df[['lowest_price', 'highest_price']].describe())

# save to csv
dish_clean_df.to_csv('dish_clean.csv', index=False)


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Null values after cleaning:
id                    0
name                  0
menus_appeared        0
times_appeared        0
first_appeared    55287
last_appeared     55284
lowest_price          0
highest_price         0
dtype: int64

Shape of cleaned dataframe: (423397, 8)

Sample of cleaned data:
   id                        name  menus_appeared  times_appeared  \
0   1  Consomme Printaniere Royal               8               8   
1   2               Chicken Gumbo             111             117   
2   3         Tomato Aux Croutons              13              13   
3   4             Onion Au Gratin              41              41   
4   5                 St. Emilion              66              68   

   first_appeared  last_appeared  lowest_price  highest_price  
0          1897.0         1927.0          0.20            0.4  
1          1895.0         1960.0          0.10            0.8  
2          1893.0         1917.0          0.25            0.4  
3          1900.0         1971

### Clean `Menu_Df`

First, let's provide an overview of the data.

In [74]:
menu_df.head()

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
0,12463,NaN,HOTEL EASTMAN,BREAKFAST,COMMERCIAL,"HOT SPRINGS, AR",CARD; 4.75X7.5;,EASTER;,NaN,1900-2822,NaN,NaN,1900-04-15,Hotel Eastman,NaN,NaN,NaN,complete,2,67
1,12464,NaN,REPUBLICAN HOUSE,[DINNER],COMMERCIAL,"MILWAUKEE, [WI];",CARD; ILLUS; COL; 7.0X9.0;,EASTER;,WEDGEWOOD BLUE CARD; WHITE EMBOSSED GREEK KEY ...,1900-2825,NaN,NaN,1900-04-15,Republican House,NaN,NaN,NaN,under review,2,34
2,12465,NaN,NORDDEUTSCHER LLOYD BREMEN,FRUHSTUCK/BREAKFAST;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, STEAMSHIP A...",1900-2827,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,84
3,12466,NaN,NORDDEUTSCHER LLOYD BREMEN,LUNCH;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2828,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,63
4,12467,NaN,NORDDEUTSCHER LLOYD BREMEN,DINNER;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,FOLDER; ILLU; COL; 5.5X7.5;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2829,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,4,33


In [75]:
menu_df.shape

(17545, 20)

In [76]:
menu_df.describe().round(2)

,id,keywords,language,location_type,page_count,dish_count
count,17545.00,0.0,0.0,0.0,17545.00,17545.00
mean,25325.95,NaN,NaN,NaN,3.48,75.62
std,6431.55,NaN,NaN,NaN,3.30,98.44
min,12463.00,NaN,NaN,NaN,1.00,0.00
25%,20742.00,NaN,NaN,NaN,2.00,20.00
50%,26165.00,NaN,NaN,NaN,2.00,35.00
75%,30707.00,NaN,NaN,NaN,4.00,93.00
max,35526.00,NaN,NaN,NaN,74.00,4053.00


In [77]:
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17545 entries, 0 to 17544
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17545 non-null  int64  
 1   name                  3197 non-null   object 
 2   sponsor               15984 non-null  object 
 3   event                 8154 non-null   object 
 4   venue                 8119 non-null   object 
 5   place                 8123 non-null   object 
 6   physical_description  14763 non-null  object 
 7   occasion              3791 non-null   object 
 8   notes                 10613 non-null  object 
 9   call_number           15983 non-null  object 
 10  keywords              0 non-null      float64
 11  language              0 non-null      float64
 12  date                  16959 non-null  object 
 13  location              17545 non-null  object 
 14  location_type         0 non-null      float64
 15  currency           

In [78]:
menu_df.nunique()

id                      17545
name                      797
sponsor                  6370
event                    1770
venue                     233
place                    3714
physical_description     6268
occasion                  423
notes                    6969
call_number             15936
keywords                    0
language                    0
date                     6599
location                 6283
location_type               0
currency                   42
currency_symbol            34
status                      2
page_count                 46
dish_count                555
dtype: int64

Some major issues here include:

-   Missing values in several columns, including 'name', 'sponsor', 'event', 'venue', 'place', 'physical_description', 'occasion', 'notes', 'call_number', 'date', 'currency', and 'currency_symbol'.
-   Completely empty columns include 'keywords', 'language', and 'location_type' (all null values).
-   Potential inconsistencies in casing, whitespace, and punctuation (not directly visible, but common issues).-   Possible outliers in numerical columns like 'page_count' and 'dish_count'.

Let's fix some of these using Pandas.

In [79]:
def standardize_event(event):
    if pd.isna(event) or not isinstance(event, str):
        return event
    event = event.lower()
    if 'breakfast' in event or 'morning' in event:
        return 'Breakfast'
    elif 'lunch' in event or 'noon' in event:
        return 'Lunch'
    elif 'dinner' in event or 'supper' in event or 'evening' in event:
        return 'Dinner'
    elif 'banquet' in event:
        return 'Banquet'
    elif 'wedding' in event:
        return 'Wedding'
    elif 'brunch' in event:
        return 'Brunch'
    elif 'birthday' in event:
        return 'Birthday'
    elif 'party' in event:
        return 'Party'
    elif 'anniversary' in event:
        return 'Anniversary'
    else:
        return event.title()

df_dropped_columns = menu_df.drop(columns=['currency_symbol', 'venue'], errors='ignore')

df_without_empty_columns = df_dropped_columns.dropna(axis=1, how='all')

df_handled_missing = df_without_empty_columns.copy()

# For categorical columns, fill with 'Unknown'
categorical_cols = ['name', 'sponsor', 'event', 'place', 'physical_description', 'occasion', 'notes', 'call_number', 'currency']
for col in categorical_cols:
    df_handled_missing[col] = df_handled_missing[col].fillna('Unknown')

# Impute 'Dollars' for missing currency
df_handled_missing['currency'] = df_handled_missing['currency'].replace('Unknown', 'Dollars')

# For date column, fill with the median date
df_handled_missing['date'] = pd.to_datetime(df_handled_missing['date'], errors='coerce')
median_date = df_handled_missing['date'].median()
df_handled_missing['date'] = df_handled_missing['date'].fillna(median_date)

def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (df[column] < lower_bound) | (df[column] > upper_bound)

df_outliers_handled = df_handled_missing.copy()
for col in ['page_count', 'dish_count']:
    outliers = detect_outliers(df_outliers_handled, col)
    df_outliers_handled.loc[outliers, col] = df_outliers_handled[col].median()

df_cleaned_text = df_outliers_handled.copy()
text_columns = ['name', 'sponsor', 'place', 'physical_description', 'occasion', 'notes', 'call_number', 'location', 'currency']
for col in text_columns:
    df_cleaned_text[col] = df_cleaned_text[col].apply(clean_text)
    df_cleaned_text[col] = df_cleaned_text[col].replace('Unknown', np.nan)

df_standardized_event = df_cleaned_text.copy()
df_standardized_event['event'] = df_standardized_event['event'].apply(standardize_event)

event_counts = df_standardized_event['event'].value_counts()
singular_events = event_counts[event_counts == 1].index
menu_clean = df_standardized_event.copy()
menu_clean.loc[menu_clean['event'].isin(singular_events), 'event'] = 'Unknown'

print("Null values after cleaning:")
print(menu_clean.isnull().sum())

print("\nShape of cleaned dataframe:", menu_clean.shape)

print("\nSample of cleaned data:")
print(menu_clean.head())

print("\nSummary statistics of numerical columns:")
print(menu_clean[['page_count', 'dish_count']].describe())

print("\nUnique values in categorical columns:")
for col in categorical_cols:
    print(f"{col}: {menu_clean[col].nunique()}")

print("\nUnique events after standardization and dropping singular events:")
print(menu_clean['event'].value_counts())

# save to csv
menu_clean.to_csv('menu_clean.csv', index=False)


Null values after cleaning:
id                          0
name                    14348
sponsor                  1576
event                       0
place                    9435
physical_description     2782
occasion                13754
notes                    6932
call_number              1562
date                        0
location                   15
currency                    0
status                      0
page_count                  0
dish_count                  0
dtype: int64

Shape of cleaned dataframe: (17545, 15)

Sample of cleaned data:
      id name                     sponsor      event  \
0  12463  NaN               Hotel Eastman  Breakfast   
1  12464  NaN            Republican House     Dinner   
2  12465  NaN  Norddeutscher Lloyd Bremen  Breakfast   
3  12466  NaN  Norddeutscher Lloyd Bremen      Lunch   
4  12467  NaN  Norddeutscher Lloyd Bremen     Dinner   

                               place       physical_description occasion  \
0                    Hot Sprin

### Clean `menu_item_df

In [80]:
menuitem_df.head()

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,1,1389,0.4,NaN,1.0,2011-03-28 15:00:44 UTC,2011-04-19 04:33:15 UTC,0.111429,0.254735
1,2,1389,0.6,NaN,2.0,2011-03-28 15:01:13 UTC,2011-04-19 15:00:54 UTC,0.438571,0.254735
2,3,1389,0.4,NaN,3.0,2011-03-28 15:01:40 UTC,2011-04-19 19:10:05 UTC,0.140000,0.261922
3,4,1389,0.5,NaN,4.0,2011-03-28 15:01:51 UTC,2011-04-19 19:07:01 UTC,0.377143,0.262720
4,5,3079,0.5,1.0,5.0,2011-03-28 15:21:26 UTC,2011-04-13 15:25:27 UTC,0.105714,0.313178


In [81]:
menuitem_df.shape

(1332726, 9)

In [82]:
menuitem_df.describe().round(2)

,id,menu_page_id,price,high_price,dish_id,xpos,ypos
count,1332726.00,1332726.00,886810.00,91905.00,1332485.00,1332726.00,1332726.00
mean,697898.38,47594.87,12.84,8.11,158011.04,0.39,0.55
std,399980.67,22039.21,499.55,90.10,167762.04,0.22,0.22
min,1.00,130.00,0.00,0.00,1.00,0.00,0.00
25%,350251.25,32049.00,0.25,0.50,5089.00,0.18,0.37
50%,702410.50,53371.00,0.40,1.25,80700.00,0.38,0.57
75%,1045548.75,66823.00,1.00,3.00,332524.00,0.57,0.74
max,1385906.00,77425.00,180000.00,7800.00,515677.00,0.99,1.00


In [83]:
menuitem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332726 entries, 0 to 1332725
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   id            1332726 non-null  int64  
 1   menu_page_id  1332726 non-null  int64  
 2   price         886810 non-null   float64
 3   high_price    91905 non-null    float64
 4   dish_id       1332485 non-null  float64
 5   created_at    1332726 non-null  object 
 6   updated_at    1332726 non-null  object 
 7   xpos          1332726 non-null  float64
 8   ypos          1332726 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 91.5+ MB


In [84]:
menuitem_df.nunique()

id              1332726
menu_page_id      26590
price              1336
high_price          671
dish_id          414138
created_at      1291090
updated_at      1295796
xpos               1323
ypos             616305
dtype: int64

Main problems here include:

1. Missing values in 'price', 'high_price', and 'dish_id' columns.
'created_at' and 'updated_at' are stored as object types instead of datetime.
2. Potential outliers or inconsistencies in 'price' and 'high_price' columns.
3. Possible duplicate entries (given the high number of unique IDs).
4. 'xpos' and 'ypos' columns might need normalization or scaling.

In [85]:
df_handled_missing = menuitem_df.copy()
# Drop rows with missing dish_id
df_handled_missing = df_handled_missing.dropna(subset=['dish_id'])
# Convert dish_id to integer
df_handled_missing['dish_id'] = df_handled_missing['dish_id'].astype(int)
# Set missing menu_page_id to null (NaN)
df_handled_missing['menu_page_id'] = df_handled_missing['menu_page_id'].astype('Int64')  # allows for NaN values
# We don't fill missing prices as they might be legitimately missing

df_datetime_converted = df_handled_missing.copy()
df_datetime_converted['created_at'] = pd.to_datetime(df_datetime_converted['created_at'], errors='coerce')
df_datetime_converted['updated_at'] = pd.to_datetime(df_datetime_converted['updated_at'], errors='coerce')

def clean_price(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df.loc[df[column] < lower_bound, column] = np.nan
    df.loc[df[column] > upper_bound, column] = np.nan
    return df

df_price_cleaned = df_datetime_converted.copy()
df_price_cleaned = clean_price(df_price_cleaned, 'price')
df_price_cleaned = clean_price(df_price_cleaned, 'high_price')

# Ensure high_price is always >= price
mask = (df_price_cleaned['high_price'].notna()) & (df_price_cleaned['price'].notna())
df_price_cleaned.loc[mask, 'high_price'] = df_price_cleaned.loc[mask, ['price', 'high_price']].max(axis=1)

df_positions_normalized = df_price_cleaned.copy()
df_positions_normalized['xpos'] = (df_positions_normalized['xpos'] - df_positions_normalized['xpos'].min()) / (df_positions_normalized['xpos'].max() - df_positions_normalized['xpos'].min())
df_positions_normalized['ypos'] = (df_positions_normalized['ypos'] - df_positions_normalized['ypos'].min()) / (df_positions_normalized['ypos'].max() - df_positions_normalized['ypos'].min())

menuitem_clean = df_positions_normalized.drop_duplicates()

print("Null values after cleaning:")
print(menuitem_clean.isnull().sum())

print("\nShape of cleaned dataframe:", menuitem_clean.shape)

print("\nSample of cleaned data:")
print(menuitem_clean.head())

print("\nSummary statistics of numerical columns:")
print(menuitem_clean.describe())

print("\nData types of columns:")
print(menuitem_clean.dtypes)

print("\nNumber of unique values in each column:")
print(menuitem_clean.nunique())

# save to csv
menuitem_clean.to_csv('menuitem_clean.csv', index=False)


<ipython-input-85-ed614d8f2b23>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_handled_missing['dish_id'] = df_handled_missing['dish_id'].astype(int)
<ipython-input-85-ed614d8f2b23>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_handled_missing['menu_page_id'] = df_handled_missing['menu_page_id'].astype('Int64')  # allows for NaN values


Null values after cleaning:
id                    0
menu_page_id          0
price            567827
high_price      1247888
dish_id               0
created_at            0
updated_at            0
xpos                  0
ypos                  0
dtype: int64

Shape of cleaned dataframe: (1332485, 9)

Sample of cleaned data:
   id  menu_page_id  price  high_price  dish_id                created_at  \
0   1          1389    0.4         NaN        1 2011-03-28 15:00:44+00:00   
1   2          1389    0.6         NaN        2 2011-03-28 15:01:13+00:00   
2   3          1389    0.4         NaN        3 2011-03-28 15:01:40+00:00   
3   4          1389    0.5         NaN        4 2011-03-28 15:01:51+00:00   
4   5          3079    0.5         1.0        5 2011-03-28 15:21:26+00:00   

                 updated_at      xpos      ypos  
0 2011-04-19 04:33:15+00:00  0.113088  0.255187  
1 2011-04-19 15:00:54+00:00  0.445099  0.255187  
2 2011-04-19 19:10:05+00:00  0.142084  0.262386  
3 2011-04-19 

### Clean `Menupage`

In [86]:
menupage_df.head()

,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99


In [87]:
menupage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66937 entries, 0 to 66936
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           66937 non-null  int64  
 1   menu_id      66937 non-null  int64  
 2   page_number  65735 non-null  float64
 3   image_id     66937 non-null  object 
 4   full_height  66608 non-null  float64
 5   full_width   66608 non-null  float64
 6   uuid         66937 non-null  object 
dtypes: float64(3), int64(2), object(2)
memory usage: 3.6+ MB


In [88]:
menupage_df.shape

(66937, 7)

In [89]:
menupage_df.describe().round(2)

,id,menu_id,page_number,full_height,full_width
count,66937.00,66937.00,65735.00,66608.00,66608.00
mean,42719.76,25653.58,3.76,3859.10,2778.59
std,21274.00,6158.83,4.91,1156.01,970.29
min,119.00,12460.00,1.00,616.00,558.00
25%,27108.00,21743.00,1.00,2988.00,2120.00
50%,43894.00,26202.00,2.00,3630.00,2527.00
75%,60696.00,30531.00,4.00,4617.25,3295.25
max,77431.00,35526.00,74.00,12044.00,9175.00


In [90]:
menupage_df.nunique()

id             66937
menu_id        19816
page_number       74
image_id       63244
full_height     5612
full_width      5041
uuid           63041
dtype: int64

This is pretty clean. However:

- We have a few missing values in columns (which we can pretty easily impute).
- Some data types (page number) look inappropriate (they should be ints).
- We can check for standard things like duplicates.

In [91]:
df_handled_missing = menupage_df.copy()

# Fill missing page_number with median and convert to int
median_page = df_handled_missing['page_number'].median()
df_handled_missing['page_number'] = df_handled_missing['page_number'].fillna(median_page).astype(int)

# For full_height and full_width, we'll use the median of the respective column
df_handled_missing['full_height'] = df_handled_missing['full_height'].fillna(df_handled_missing['full_height'].median())
df_handled_missing['full_width'] = df_handled_missing['full_width'].fillna(df_handled_missing['full_width'].median())

df_correct_types = df_handled_missing.copy()
# Ensure page_number is int, full_height and full_width remain float
df_correct_types['page_number'] = df_correct_types['page_number'].astype(int)
df_correct_types['full_height'] = df_correct_types['full_height'].astype(float)
df_correct_types['full_width'] = df_correct_types['full_width'].astype(float)

menupage_clean = df_correct_types.drop_duplicates()

print("Null values after cleaning:")
print(menupage_clean.isnull().sum())

print("\nShape of cleaned dataframe:", menupage_clean.shape)

print("\nSample of cleaned data:")
print(menupage_clean.head())

print("\nSummary statistics of numerical columns:")
print(menupage_clean.describe())

print("\nData types of columns:")
print(menupage_clean.dtypes)

print("\nNumber of unique values in each column:")
print(menupage_clean.nunique())

# save to csv
menupage_clean.to_csv('menupage_clean.csv', index=False)

Null values after cleaning:
id             0
menu_id        0
page_number    0
image_id       0
full_height    0
full_width     0
uuid           0
dtype: int64

Shape of cleaned dataframe: (66937, 7)

Sample of cleaned data:
    id  menu_id  page_number image_id  full_height  full_width  \
0  119    12460            1  1603595       7230.0      5428.0   
1  120    12460            2  1603596       5428.0      7230.0   
2  121    12460            3  1603597       7230.0      5428.0   
3  122    12460            4  1603598       7230.0      5428.0   
4  123    12461            1  1603591       7230.0      5428.0   

                                   uuid  
0  510d47e4-2955-a3d9-e040-e00a18064a99  
1  510d47e4-2956-a3d9-e040-e00a18064a99  
2  510d47e4-2957-a3d9-e040-e00a18064a99  
3  510d47e4-2958-a3d9-e040-e00a18064a99  
4  510d47e4-2959-a3d9-e040-e00a18064a99  

Summary statistics of numerical columns:
                 id       menu_id   page_number   full_height    full_width
count  6

## Load Data Into SQL Database

In [92]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('nypl_menus.db')

dish_clean_df.to_sql('dishes', conn, if_exists='replace', index=False)
print("Dishes data loaded successfully.")

menu_clean.to_sql('menus', conn, if_exists='replace', index=False)
print("Menus data loaded successfully.")

menuitem_clean.to_sql('menuitems', conn, if_exists='replace', index=False)
print("Menu items data loaded successfully.")

menupage_clean.to_sql('menupages', conn, if_exists='replace', index=False)
print("Menu pages data loaded successfully.")

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database created and data loaded successfully.")

# Reopen the connection to verify the data
conn = sqlite3.connect('nypl_menus.db')
cursor = conn.cursor()

# Check the number of rows in each table
tables = ['dishes', 'menus', 'menuitems', 'menupages']
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"Number of rows in {table} table: {count}")

conn.close()


Dishes data loaded successfully.
Menus data loaded successfully.
Menu items data loaded successfully.
Menu pages data loaded successfully.
Database created and data loaded successfully.
Number of rows in dishes table: 423397
Number of rows in menus table: 17545
Number of rows in menuitems table: 1332485
Number of rows in menupages table: 66937


Now, let's use SQL magic to inspect this:

In [93]:
%reload_ext sql
%sql sqlite:///nypl_menus.db

In [94]:
%%sql
SELECT * FROM dishes LIMIT 5;

   sqlite:///dirty_menus.db
 * sqlite:///nypl_menus.db
Done.


id,name,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
1,Consomme Printaniere Royal,8,8,1897.0,1927.0,0.2,0.4
2,Chicken Gumbo,111,117,1895.0,1960.0,0.1,0.8
3,Tomato Aux Croutons,13,13,1893.0,1917.0,0.25,0.4
4,Onion Au Gratin,41,41,1900.0,1971.0,0.25,1.0
5,St. Emilion,66,68,1881.0,1981.0,0.0,0.0


In [95]:
%%sql
SELECT * FROM menus LIMIT 5;

   sqlite:///dirty_menus.db
 * sqlite:///nypl_menus.db
Done.


id,name,sponsor,event,place,physical_description,occasion,notes,call_number,date,location,currency,status,page_count,dish_count
12463,None,Hotel Eastman,Breakfast,"Hot Springs, Ar",Card 4. 75X7. 5,Easter,None,1900-2822,1900-04-15 00:00:00,Hotel Eastman,Dollars,complete,2,67
12464,None,Republican House,Dinner,"Milwaukee, Wi",Card Illus Col 7. 0X9. 0,Easter,Wedgewood Blue Card White Embossed Greek Key Border Easter Sunday Embossed In White Violet Colored Spray Of Flowers In Upper Left Corner,1900-2825,1900-04-15 00:00:00,Republican House,Dollars,under review,2,34
12465,None,Norddeutscher Lloyd Bremen,Breakfast,Dampfer Kaiser Wilhelm Der Grosse,Card Illu Col 5. 5X8. 0,None,"Menu In German And English Illus, Steamship And Sailing Vessel",1900-2827,1900-04-16 00:00:00,Norddeutscher Lloyd Bremen,Dollars,complete,2,84
12466,None,Norddeutscher Lloyd Bremen,Lunch,Dampfer Kaiser Wilhelm Der Grosse,Card Illu Col 5. 5X8. 0,None,"Menu In German And English Illus, Harbor Scene With Sailing Vessel",1900-2828,1900-04-16 00:00:00,Norddeutscher Lloyd Bremen,Dollars,complete,2,63
12467,None,Norddeutscher Lloyd Bremen,Dinner,Dampfer Kaiser Wilhelm Der Grosse,Folder Illu Col 5. 5X7. 5,None,"Menu In German And English Illus, Harbor Scene With Rocks And Lighthouse Steamship And Sailing Vessels Concert Program Dates On German Side Of Menu Montag, Den 16 April 1900 On English Side Of Menu Monday, April 15Th, 1900",1900-2829,1900-04-16 00:00:00,Norddeutscher Lloyd Bremen,Dollars,complete,4,33


In [96]:
%%sql
SELECT * FROM menuitems LIMIT 5;

   sqlite:///dirty_menus.db
 * sqlite:///nypl_menus.db
Done.


id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
1,1389,0.4,None,1,2011-03-28 15:00:44+00:00,2011-04-19 04:33:15+00:00,0.11308765666023568,0.2551866804243511
2,1389,0.6,None,2,2011-03-28 15:01:13+00:00,2011-04-19 15:00:54+00:00,0.44509927100787244,0.2551866804243511
3,1389,0.4,None,3,2011-03-28 15:01:40+00:00,2011-04-19 19:10:05+00:00,0.14208394522460935,0.26238642397042766
4,1389,0.5,None,4,2011-03-28 15:01:51+00:00,2011-04-19 19:07:01+00:00,0.382756895384606,0.2631858389349148
5,3079,0.5,1.0,5,2011-03-28 15:21:26+00:00,2011-04-13 15:25:27+00:00,0.10728758703910252,0.31373330795508053


In [97]:
%%sql
SELECT * FROM menupages LIMIT 5;

   sqlite:///dirty_menus.db
 * sqlite:///nypl_menus.db
Done.


id,menu_id,page_number,image_id,full_height,full_width,uuid
119,12460,1,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
120,12460,2,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
121,12460,3,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
122,12460,4,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
123,12461,1,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99


## Workflow Graphic (In mermaid)

In [98]:
mm("""
graph TD
    A[Raw Data Download] --> B[Unzip Files]
    B --> C{Split into 4 Datasets}
    C -->|Dish Data| D[dish_df]
    C -->|Menu Data| E[menu_df]
    C -->|MenuItem Data| F[menuitem_df]
    C -->|MenuPage Data| G[menupage_df]

    D --> H[Clean dish_df]
    H --> |Drop null columns| H1[df_without_null_columns]
    H1 --> |Handle outliers| H2[dish_clean.csv]

    E --> I[Clean menu_df]
    I --> |Drop empty columns| I1[df_without_empty_columns]
    I1 --> |Handle missing values| I2[df_handled_missing]
    I2 --> |Handle outliers| I3[df_outliers_handled]
    I3 --> |Clean text| I4[df_cleaned_text]
    I4 --> |Standardize events| I5[df_standardized_event]
    I5 --> |Handle singular events| I6[menu_clean.csv]

    F --> J[Clean menuitem_df]
    J --> |Handle missing values| J1[df_handled_missing]
    J1 --> |Convert datetime| J2[df_datetime_converted]
    J2 --> |Clean prices| J3[df_price_cleaned]
    J3 --> |Normalize positions| J4[df_positions_normalized]
    J4 --> |Drop duplicates| J5[menuitem_clean.csv]

    G --> K[Clean menupage_df]
    K --> |Handle missing values| K1[df_handled_missing]
    K1 --> |Correct data types| K2[df_correct_types]
    K2 --> |Drop duplicates| K3[menupage_clean.csv]

    H2 --> L[Load into SQLite]
    I6 --> L
    J5 --> L
    K3 --> L

    L --> M[dishes table]
    L --> N[menus table]
    L --> O[menuitems table]
    L --> P[menupages table]

    M --> Q[Final SQLite Database]
    N --> Q
    O --> Q
    P --> Q

"""
)